In [12]:
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re 
import pandas as pd 

In [2]:
html = urlopen('https://www.khidi.or.kr/kps/dhraStat/result9?menuId=MENU01662&gubun=ageall&year=2008')
bsobj=BeautifulSoup(html, 'lxml')

In [3]:
age = []
for i in bsobj.find('ul',{'id':'subTabMenu', \
                          'class':'normal_tab'}).findAll('a'):
    age.append(re.findall('(.*)세',i.text)[0])

In [13]:
KR_foodGIntake = list()

In [16]:
for gender in ['man','all','woman']: 
    gubun = 'gubun=age'+gender
    
    for yr in range(2008,2019): 
        year = '&year='+str(yr)
        
        for a in age:

            ageGroup = '&category='+a
            
            url = 'https://www.khidi.or.kr/kps/dhraStat/result9?menuId=MENU01662&'+gubun+year+ageGroup
            
            html = urlopen(url)
            bsobj=BeautifulSoup(html, 'lxml')
            
            foodGIntake = [i.text for i in bsobj.find('tbody').findAll('td')]
            
            p = dict()
            
            for i in range(1,len(foodGIntake),12):
                p[foodGIntake[i-1]] = [foodGIntake[i]]
            
            p['Year'] = [yr]
            p['Age'] = a
                
            if gender == 'man': 
                p['Gender'] = 'Male'
            elif gender == 'woman':
                p['Gender'] = 'Female'
            elif gender == 'all':
                p['Gender'] = 'All'

            KR_foodGIntake.append(pd.DataFrame(p))

In [18]:
finalTB = pd.concat(KR_foodGIntake)

In [34]:
finalTB.columns

Index(['곡류 및 그 제품', '감자 및 전분류', '당류 및 그 제품', '두류 및 그 제품', '종실류 및 그 제품', '채소류',
       '버섯류', '과실류', '해조류', '음료 및 주류', '조미료류', '유지류(식물성)', '기타(식물성)',
       '육류 및 그 제품', '난류', '어패류', '우유류 및 그 제품', '유지류(동물성)', 'Year', 'Age',
       'Gender', '유지(식물성)', '유류 및 그 제품', '유지(동물성)', '기타(동물성)', '음료류', '주류'],
      dtype='object')

In [49]:
finalTB['우유류 및 그 제품'][finalTB['우유류 및 그 제품'].isna()] = \
finalTB['유류 및 그 제품'][finalTB['우유류 및 그 제품'].isna()] 

/Users/dahyekim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [60]:
finalTB['유지류(식물성)'][finalTB['유지류(식물성)'].isna()] =\
finalTB['유지(식물성)'][finalTB['유지류(식물성)'].isna()]

/Users/dahyekim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [61]:
finalTB['유지류(동물성)'][finalTB['유지류(동물성)'].isna()] =\
finalTB['유지(동물성)'][finalTB['유지류(동물성)'].isna()]

/Users/dahyekim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [67]:
finalTB['음료 및 주류'] = finalTB['음료 및 주류'].astype(float)
finalTB['음료류'] = finalTB['음료류'].astype(float)
finalTB['주류'] = finalTB['주류'].astype(float)

In [69]:
finalTB['음료 및 주류'][finalTB['음료 및 주류'].isna()] = \
finalTB['음료류'][finalTB['음료 및 주류'].isna()] + \
finalTB['주류'][finalTB['음료 및 주류'].isna()]

/Users/dahyekim/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
finalTB.drop(['유지(동물성)','유지(식물성)','유류 및 그 제품','음료류','주류','기타(동물성)'], axis = 1, inplace = True)

In [76]:
finalTB.head(20)

,곡류 및 그 제품,감자 및 전분류,당류 및 그 제품,두류 및 그 제품,종실류 및 그 제품,채소류,버섯류,과실류,해조류,음료 및 주류,...,유지류(식물성),기타(식물성),육류 및 그 제품,난류,어패류,우유류 및 그 제품,유지류(동물성),Year,Age,Gender
0,145.61,20.67,1.91,43.03,0.76,63.4,2.17,151.69,1.75,13.53,...,2.94,0.26,41.05,15.91,14.95,215.92,0.04,2008,1-2,Male
0,201.74,19.55,4.37,27.91,2.07,97.03,2.81,135.97,5.04,28.05,...,4.15,0.08,34.84,22.24,19.55,247.58,0.11,2008,3-5,Male
0,289.49,32.11,6.05,27.83,1.63,185.94,3.42,161.84,2.93,58.69,...,6.96,0.11,83.36,28.56,38.31,220.79,0.22,2008,6-11,Male
0,364.19,42.34,7,27.64,2.31,256.66,5.01,98.84,3.67,124.98,...,9.02,0.26,118.27,33.73,36.69,175.97,0.47,2008,12-18,Male
0,309.31,35.75,10.79,45.09,2.27,313.91,3.33,110.15,4.21,321.58,...,12.29,0.19,162.55,31.21,76,124.83,0.29,2008,19-29,Male
0,331.3,32.65,10.67,43.32,2.82,415.03,3.85,160.87,6.11,308.01,...,11.15,0.42,115.56,30.31,78.24,61.58,0.19,2008,30-49,Male
0,335.58,36.83,9,52.13,3.96,413.4,3.37,160.34,5.86,205.95,...,8.29,0.21,83.77,16.5,69.96,43.98,0.04,2008,50-64,Male
0,303.88,33.06,4.94,41.91,2.39,338.97,3.39,124.87,4.37,128.38,...,4.04,0.76,54.82,10.19,44.96,36.1,NaN,2008,65,Male
0,126.9,24.1,3.3,45.4,0.7,50.2,2.6,128.8,1.9,17.10,...,2.4,0.5,21,20.5,14.4,272.5,0,2009,1-2,Male
0,199.7,25.8,5.2,22.4,1,101.5,3.5,152.8,3.7,33.50,...,4.5,0.9,42,30.9,23.5,226.1,0.2,2009,3-5,Male


In [77]:
finalTB.to_csv('finalTB_beforeWrangling.csv', index = False)